In [2]:
import os
import re
import glob 
import json
import random
import shutil
import numpy as np
import shapeworks as sw
import SimpleITK as sitk
import pandas as pd

from tqdm import tqdm
from pathlib import Path 
from scipy.ndimage import label, binary_fill_holes, binary_opening

In [3]:
nrrd_dir = 'nrrd segmentations/'

data_dir = "data/"
muscle_name = 'vastus_medialis'  # Change this to the desired muscle name
muscle_dir = f"{data_dir}{muscle_name}/"
bin_dir = f"{muscle_dir}bin/"
mesh_dir = f"{muscle_dir}mesh/"
mesh_rig_dir = f"{muscle_dir}mesh_rigid/"
mesh_novol_dir = f"{muscle_dir}mesh_novolume/"
input_mesh2ssm = f"{muscle_dir}input_data/"

os.makedirs(data_dir, exist_ok=True)
os.makedirs(muscle_dir, exist_ok=True)
os.makedirs(bin_dir, exist_ok=True)
os.makedirs(mesh_dir, exist_ok=True)
os.makedirs(mesh_rig_dir, exist_ok=True)
os.makedirs(mesh_novol_dir, exist_ok=True)

Extensor Group:
- vastus_lateralis
- vastus_medialis
- vastus_intermedius
- rectus_femoris

Primary Flexor Group:
- biceps_femoris_long
- Biceps_femoris_short
- semitendinosus
- semimembranosus

# Function

In [4]:
# -------- Load Images --------
def load_images(folder, left=True, right=True):
    all_files = sorted(glob.glob(os.path.join(folder, '*' + '.nrrd')))
    left_files = [f for f in all_files if "_left" in os.path.basename(f)] if left else []
    right_files = [f for f in all_files if "_right" in os.path.basename(f)] if right else []

    # Calculate how many to take from each
    if left and right:
        binary_filenames = all_files
    elif left:
        binary_filenames = left_files
    elif right:
        binary_filenames = right_files
    else:
        raise ValueError("At least one of 'left' or 'right' must be True.")
    return [sitk.ReadImage(f) for f in binary_filenames]

def downsample_image(image, factor=2):
    original_size = np.array(image.GetSize())
    original_spacing = np.array(image.GetSpacing())
    
    new_size = (original_size // factor).astype(int)
    new_spacing = original_spacing * (original_size / new_size)

    resampled = sitk.Resample(image,
                              size=[int(sz) for sz in new_size],
                              transform=sitk.Transform(),
                              interpolator=sitk.sitkLinear,
                              outputOrigin=image.GetOrigin(),
                              outputSpacing=[float(sp) for sp in new_spacing],
                              outputDirection=image.GetDirection(),
                              defaultPixelValue=0.0)
    return resampled


# -------- Distance Map Conversion --------
def to_distance_map(img, clip_val=500):
    img_bin = sitk.Cast(img, sitk.sitkUInt8)
    dist = sitk.SignedMaurerDistanceMap(img_bin, insideIsPositive=True, useImageSpacing=True)
    dist = sitk.Clamp(dist, lowerBound=-clip_val, upperBound=clip_val)
    return dist

# -------- Masked Mean Calculation --------
def masked_mean(images, clip_val=500):
    arrays = [np.clip(sitk.GetArrayFromImage(img), -clip_val, clip_val) for img in images]
    stacked = np.stack(arrays, axis=0)
    mask = np.any(stacked != 0, axis=0)
    sum_img = np.sum(stacked, axis=0)
    count = np.count_nonzero(stacked != 0, axis=0)
    count[count == 0] = 1
    avg = sum_img / count
    avg[~mask] = 0
    mean_img = sitk.GetImageFromArray(avg)
    mean_img.CopyInformation(images[0])
    return mean_img

# -------- Apply Transform --------
def apply_transform(moving, reference, transform):
    return sitk.Resample(moving, reference, transform, sitk.sitkLinear, 0.0, moving.GetPixelID())

# -------- Rigid Registration --------
def rigid_register(fixed, moving):
    initial_transform = sitk.Euler3DTransform(
        sitk.CenteredTransformInitializer(
            fixed, moving,
            sitk.Euler3DTransform(),
            sitk.CenteredTransformInitializerFilter.MOMENTS
        )
    )
    initial_transform.SetRotation(0.0, 0.0, 0.0)
    registration = sitk.ImageRegistrationMethod()
    registration.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)

    # 🧠 Conservative optimizer settings
    registration.SetOptimizerAsGradientDescent(
        learningRate=0.05,               # still conservative
        numberOfIterations=200,         # fewer needed with pyramid
        convergenceMinimumValue=1e-6,
        convergenceWindowSize=10
    )
    registration.SetShrinkFactorsPerLevel([4, 2, 1])
    registration.SetSmoothingSigmasPerLevel([2, 1, 0])
    registration.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    registration.SetInterpolator(sitk.sitkLinear)
    registration.SetInitialTransform(initial_transform, inPlace=False)
    
    final_transform = registration.Execute(fixed, moving)
    return final_transform


# -------- Affine Registration --------
def affine_register(fixed, moving):
    tx = sitk.AffineTransform(3)  # Identity by default

    # Ensure both are Float32
    fixed = sitk.Cast(fixed, sitk.sitkFloat32)
    moving = sitk.Cast(moving, sitk.sitkFloat32)

    # 🧠 Set transform center to fixed image center (in physical space)
    center = fixed.TransformContinuousIndexToPhysicalPoint(
        [sz // 2 for sz in fixed.GetSize()]
    )
    tx.SetCenter(center)

    registration = sitk.ImageRegistrationMethod()
    registration.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)

    registration.SetOptimizerAsGradientDescent(
        learningRate=0.01,
        numberOfIterations=200,
        convergenceMinimumValue=1e-7,
        convergenceWindowSize=20
    )
    registration.SetOptimizerScalesFromIndexShift()
    registration.SetShrinkFactorsPerLevel([4, 2, 1])
    registration.SetSmoothingSigmasPerLevel([2, 1, 0])
    registration.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    registration.SetInterpolator(sitk.sitkLinear)
    registration.SetInitialTransform(tx, inPlace=False)

    final_transform = registration.Execute(fixed, moving)
    return final_transform

# -------- BSpline Registration --------
def bspline_register(fixed, moving, num_ctrl_pts=8):
    # Ensure both are Float32
    fixed = sitk.Cast(downsample_image(fixed), sitk.sitkFloat32)
    moving = sitk.Cast(downsample_image(moving), sitk.sitkFloat32)

    grid_physical_spacing = [(sz - 1) * sp / (num_ctrl_pts - 3)
                             for sz, sp in zip(fixed.GetSize(), fixed.GetSpacing())]
    mesh_size = [int(sz / sp + 0.5) for sz, sp in zip(fixed.GetSize(), grid_physical_spacing)]
    transform = sitk.BSplineTransformInitializer(fixed, mesh_size, order=3)

    registration = sitk.ImageRegistrationMethod()
    registration.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration.SetOptimizerAsLBFGSB(gradientConvergenceTolerance=1e-5, numberOfIterations=100)
    registration.SetInterpolator(sitk.sitkLinear)
    registration.SetInitialTransform(transform, inPlace=False)

    registration.SetShrinkFactorsPerLevel([4, 2, 1])
    registration.SetSmoothingSigmasPerLevel([2, 1, 0])
    registration.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    final_transform = registration.Execute(fixed, moving)
    return final_transform

def check_overlap(fixed, moving, threshold=0.01):
    """Check if bounding boxes of fixed and moving overlap sufficiently."""
    fixed_arr = sitk.GetArrayFromImage(fixed) != 0
    moving_arr = sitk.GetArrayFromImage(moving) != 0

    if np.sum(moving_arr) == 0 or np.sum(fixed_arr) == 0:
        return False

    overlap = np.logical_and(fixed_arr, moving_arr)
    overlap_ratio = np.sum(overlap) / np.sum(moving_arr)

    return overlap_ratio > threshold

In [5]:
def sitk_to_sw(sitk_img):
    arr = sitk.GetArrayFromImage(sitk_img)
    sw_img = sw.Image(arr)

    # Transfer geometry
    spacing = sitk_img.GetSpacing()
    origin = sitk_img.GetOrigin()
    direction = sitk_img.GetDirection()

    sw_img.setSpacing(spacing)
    sw_img.setOrigin(origin)

    # Direction (3×3) to CoordSys (flattened)
    sw_img.setCoordSys(direction)

    return sw_img


In [6]:
def resample_to_spacing(image, target_spacing, interpolator=sitk.sitkNearestNeighbor):
    """
    Resample an image to a new spacing (upsample or downsample).
    Keeps origin, direction, and uses a default transform (identity).
    """
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    
    new_size = [
        int(round(original_size[i] * (original_spacing[i] / target_spacing[i])))
        for i in range(3)
    ]

    return sitk.Resample(
        image,
        new_size,
        sitk.Transform(),
        interpolator,
        image.GetOrigin(),
        target_spacing,
        image.GetDirection(),
        0,
        image.GetPixelID()
    )


In [7]:
def clean_binary_volume(image: sw.Image, opening_radius=1) -> sw.Image:
    """
    Cleans a binary volume:
    - Removes small connections (via binary opening)
    - Keeps only the largest connected component
    - Fills internal holes
    """

    arr = image.toArray().astype(bool)

    # Break thin bridges
    structure = np.ones((3, 3, 3)) if opening_radius == 1 else None
    arr = binary_opening(arr, structure=structure)

    # Keep only the largest connected component
    labeled, num_features = label(arr)
    if num_features > 1:
        sizes = np.bincount(labeled.ravel())
        sizes[0] = 0  # background
        largest_label = sizes.argmax()
        arr = (labeled == largest_label)

    # Fill internal holes
    arr_filled = binary_fill_holes(arr).astype(np.float32)

    # Convert back to ShapeWorks image
    cleaned_img = sw.Image(arr_filled)
    cleaned_img.setOrigin(image.origin())
    cleaned_img.setSpacing(image.spacing())
    cleaned_img.setCoordsys(image.coordsys())

    return cleaned_img


In [8]:
def parse_label_file(filepath):
    label_dict = {}

    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            # Skip comments and empty lines
            if not line or line.startswith('#'):
                continue

            parts = line.split()
            try:
                # First part is the label index
                label = int(parts[0])
                # Label description is the rest inside the last pair of quotes
                label_name = line.split('"')[-2]
                label_dict[label] = label_name
            except (IndexError, ValueError):
                continue  # skip malformed lines

    return label_dict

def normalize_name(name):
    """Lowercase, remove non-letters and digits, then strip leading numbers."""
    name = name.lower()
    name = re.sub(r'[^a-z]+', '', name)  # keep only letters
    return name

def get_labels(muscle_name):
    normalized_target = normalize_name(muscle_name)

    left_label = None
    right_label = None

    label_file = "Muscles_leg_labels.txt"
    label_dict = parse_label_file(label_file)

    for label, name in label_dict.items():
        if label in (0, 151):  # Skip unwanted labels
            continue

        norm = normalize_name(name)

        if normalized_target in norm:
            if 'left' in name.lower():
                left_label = label
            elif 'right' in name.lower():
                right_label = label

    if left_label is None or right_label is None:
        raise ValueError(f"Could not find both left and right labels for '{muscle_name}'")

    return (left_label, right_label)

In [9]:
# Compute volume for volume-normalized
def compute_mesh_volume(mesh):
    """
    Compute signed volume of a closed triangular mesh in mm^3.
    """
    pts = mesh.points()
    faces = mesh.faces()
    v1 = pts[faces[:, 0]]
    v2 = pts[faces[:, 1]]
    v3 = pts[faces[:, 2]]
    vol = np.sum(np.einsum('ij,ij->i', v1, np.cross(v2, v3))) / 6.0
    return abs(vol)

# Extract label

In [10]:
# Load and initial grooming
shape_extention = '.nrrd'
random.seed(41)  # For reproducibility

# Files to process
all = True
left = True  # Process left segmentations
right = True # Process right segmentations

num_samples = 30
randomize = False  # Randomize the order of files

left_label, right_label = get_labels(muscle_name)
print(f"Processing muscle: {muscle_name}")
print(f"Left label: {left_label}, Right label: {right_label}")

shape_filenames = sorted(glob.glob(nrrd_dir + '*' + shape_extention))
if randomize:
    random.shuffle(shape_filenames)

if not all:
    shape_filenames = shape_filenames[:num_samples]
print ('Number of shapes: ' + str(len(shape_filenames)))
print('Shape files found:')
for shapeFilename in shape_filenames:
    shapeFilename = Path(shapeFilename)
    print(shapeFilename)

Processing muscle: vastus_medialis
Left label: 59, Right label: 60
Number of shapes: 101
Shape files found:
nrrd segmentations/sub-MOTION001_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION002_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION003_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION004_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION005_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION006_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION007_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION008_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION009_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION010_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION011_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION012_ses-001_seg_auto_megre_dix_outph.nrrd
nrrd segmentations/sub-MOTION013_ses-001

In [11]:
# Initialize lists for storing segments and names
shape_seg_list = []
shape_names = []

# Parameters
iso_spacing = [1.5, 1.5, 1.5]  # isotropic spacing
pad_amount = 30
antialias_iterations = 30

for shape_filename in shape_filenames:
    seg_filename = shape_filename.split('/')[-1]
    shape_name = seg_filename.split('_')[0].replace('sub-', '')

    shape_seg = sw.Image(shape_filename)
    print(f"Processing shape: {shape_name}")

    if randomize:
        # Randomly select a segment to process
        choice = random.choice([0, 1])
        if choice == 0:
            left = True
            right = False
        elif choice == 1:
            left = False
            right = True
        else:
            left = True
            right = True

    # --- Left segment (label 5) ---
    if left:
        seg = shape_seg.copy().binarize(left_label, left_label)
        seg = seg.antialias(antialias_iterations)
        seg = seg.resample(iso_spacing, sw.InterpolationType.NearestNeighbor)
        seg.translate(seg.center() - seg.centerOfMass()).binarize().recenter()
        bbox = sw.ImageUtils.boundingBox([seg], 0.5).pad(pad_amount)
        seg.crop(bbox)
        seg = clean_binary_volume(seg)
        seg.setSpacing(iso_spacing)
        seg.setOrigin([0.0, 0.0, 0.0])
        
        shape_seg_list.append(seg)
        shape_names.append(f"{shape_name}_left")
        out_shape_filename = os.path.join(bin_dir, f"{shape_name}_left" + '_bin.nrrd')
        seg.write(out_shape_filename)

        print(f" Processed left segment: {out_shape_filename}")

    # --- Right segment (label 6) ---
    if right:
        seg = shape_seg.copy().binarize(right_label, right_label)
        seg = seg.antialias(antialias_iterations)
        seg = seg.resample(iso_spacing, sw.InterpolationType.NearestNeighbor)
        
        # Flip the right segment
        seg_array = np.flip(seg.toArray(), axis=2).copy()
        seg = sw.Image(seg_array)

        seg.setOrigin(seg.origin())
        seg.translate(seg.center() - seg.centerOfMass()).binarize().recenter()
        bbox = sw.ImageUtils.boundingBox([seg], 0.5).pad(pad_amount)
        seg.crop(bbox)
        seg = clean_binary_volume(seg)
        seg.setSpacing(iso_spacing)
        seg.setOrigin([0.0, 0.0, 0.0])

        shape_seg_list.append(seg)
        shape_names.append(f"{shape_name}_right")
        out_shape_filename = os.path.join(bin_dir, f"{shape_name}_right" + '_bin.nrrd')
        seg.write(out_shape_filename)

        print(f" Processed right segment: {out_shape_filename}")

print(f"Total number of shapes processed: {len(shape_seg_list)}")

Processing shape: MOTION001
 Processed left segment: data/vastus_medialis/bin/MOTION001_left_bin.nrrd
 Processed right segment: data/vastus_medialis/bin/MOTION001_right_bin.nrrd
Processing shape: MOTION002
 Processed left segment: data/vastus_medialis/bin/MOTION002_left_bin.nrrd
 Processed right segment: data/vastus_medialis/bin/MOTION002_right_bin.nrrd
Processing shape: MOTION003
 Processed left segment: data/vastus_medialis/bin/MOTION003_left_bin.nrrd
 Processed right segment: data/vastus_medialis/bin/MOTION003_right_bin.nrrd
Processing shape: MOTION004
 Processed left segment: data/vastus_medialis/bin/MOTION004_left_bin.nrrd
 Processed right segment: data/vastus_medialis/bin/MOTION004_right_bin.nrrd
Processing shape: MOTION005
 Processed left segment: data/vastus_medialis/bin/MOTION005_left_bin.nrrd
 Processed right segment: data/vastus_medialis/bin/MOTION005_right_bin.nrrd
Processing shape: MOTION006
 Processed left segment: data/vastus_medialis/bin/MOTION006_left_bin.nrrd
 Process

In [12]:
# Visualization
'''
# define parameters that controls the plotter
use_same_window = False # plot using multiple rendering windows if false
notebook        = False # True will enable the plots to lie inline
show_borders    = False  # show borders for each rendering window
shade_volumes   = True  # use shading when performing volume rendering
color_map       = "magma" # color map for volume rendering, e.g., 'bone', 'coolwarm', 'cool', 'viridis', 'magma'
show_axes       = False  # show a vtk axes widget for each rendering window
show_bounds     = False  # show volume bounding box
show_all_edges  = False  # add an unlabeled and unticked box at the boundaries of plot. 
font_size       = 10    # text font size for windows
link_views      = True  # link all rendering windows so that they share same camera and axes boundaries

# plot all segmentations in the shape list
sw.plot_volumes(shape_seg_list[:12]],   
             use_same_window = use_same_window,
             notebook        = notebook,
             show_borders    = show_borders,  
             shade_volumes   = shade_volumes, 
             color_map       = color_map,
             show_axes       = show_axes,  
             show_bounds     = show_bounds,
             show_all_edges  = show_all_edges, 
             font_size       = font_size,   
             link_views      = link_views
             )
'''

'\n# define parameters that controls the plotter\nuse_same_window = False # plot using multiple rendering windows if false\nnotebook        = False # True will enable the plots to lie inline\nshow_borders    = False  # show borders for each rendering window\nshade_volumes   = True  # use shading when performing volume rendering\ncolor_map       = "magma" # color map for volume rendering, e.g., \'bone\', \'coolwarm\', \'cool\', \'viridis\', \'magma\'\nshow_axes       = False  # show a vtk axes widget for each rendering window\nshow_bounds     = False  # show volume bounding box\nshow_all_edges  = False  # add an unlabeled and unticked box at the boundaries of plot. \nfont_size       = 10    # text font size for windows\nlink_views      = True  # link all rendering windows so that they share same camera and axes boundaries\n\n# plot all segmentations in the shape list\nsw.plot_volumes(shape_seg_list[:12]],   \n             use_same_window = use_same_window,\n             notebook        

# Find Reference

In [13]:
# Load and initial grooming
shape_extention = '.nrrd'
random.seed(41)  # For reproducibility

# Files to process
all = False
left = True  # Process left segmentations
right = False # Process right segmentations

num_samples = 101
randomize = True  # Randomize the order of files

# File names for the next step
bin_extension = '.nrrd'
all_files = sorted(glob.glob(os.path.join(bin_dir, '*' + bin_extension)))
print(f"Total number of binary files found: {len(all_files)}")

if all:
    binary_filenames = all_files
else:
    # Filter by left/right based on filename
    left_files = [f for f in all_files if "_left" in os.path.basename(f)] if left else []
    right_files = [f for f in all_files if "_right" in os.path.basename(f)] if right else []

    # Calculate how many to take from each
    if left and right:
        each_side = num_samples // 2
        selected_left = random.sample(left_files, min(each_side, len(left_files)))
        selected_right = random.sample(right_files, min(each_side, len(right_files)))
        binary_filenames = selected_left + selected_right

    elif left:
        binary_filenames = random.sample(left_files, min(num_samples, len(left_files)))

    elif right:
        binary_filenames = random.sample(right_files, min(num_samples, len(right_files)))

    else:
        raise ValueError("At least one of 'left' or 'right' must be True.")

print ('Number of binary shapes to process: ' + str(len(binary_filenames)))

Total number of binary files found: 202
Number of binary shapes to process: 101


In [14]:
# Initialize lists for storing segments and names
shape_seg_list = []
shape_names = []

for filename in binary_filenames:
    seg_filename = filename.split('/')[-1]
    shape_name = os.path.splitext(seg_filename)[0].replace('_bin', '')
    
    print(f"Processing: {shape_name}")
    shape_seg = sw.Image(filename)

    shape_seg_list.append(shape_seg)
    shape_names.append(shape_name)
    
print(f"Total segments processed: {len(shape_seg_list)}")

Processing: MOTION049_left
Processing: MOTION043_left
Processing: MOTION030_left
Processing: MOTION022_left
Processing: MOTION050_left
Processing: MOTION074_left
Processing: MOTION089_left
Processing: MOTION037_left
Processing: MOTION071_left
Processing: MOTION036_left
Processing: MOTION097_left
Processing: MOTION096_left
Processing: MOTION002_left
Processing: MOTION032_left
Processing: MOTION085_left
Processing: MOTION003_left
Processing: MOTION057_left
Processing: MOTION020_left
Processing: MOTION084_left
Processing: MOTION041_left
Processing: MOTION098_left
Processing: MOTION033_left
Processing: MOTION075_left
Processing: MOTION008_left
Processing: MOTION016_left
Processing: MOTION076_left
Processing: MOTION005_left
Processing: MOTION056_left
Processing: MOTION094_left
Processing: MOTION028_left
Processing: MOTION010_left
Processing: MOTION047_left
Processing: MOTION061_left
Processing: MOTION017_left
Processing: MOTION086_left
Processing: MOTION077_left
Processing: MOTION013_left
P

In [15]:
# 1: Downsample for fast reference selection
downsampled_list = [img.copy().resample([6, 6, 6], sw.InterpolationType.NearestNeighbor) for img in shape_seg_list]

# 2: Find reference index based on downsampled data
ref_index = sw.find_reference_image_index(downsampled_list)
print(f"Using reference: {ref_index}. {shape_names[ref_index]}")

Using reference: 22. MOTION075_left


# Template

In [16]:
# ---- SETTINGS ----
# ref_index = 16 # 12 (soleus), 123 (vastus lateralis, tibialis anterior), 17 (biceps femoris long)
clip_val = 500

# ---- STEP 1: Load and Convert ----
binaries = load_images(bin_dir, left=True, right=False)
distance_maps = [to_distance_map(img, clip_val=clip_val) for img in binaries]

rigid_mean_path = os.path.join(muscle_dir, "rigid_mean.nrrd")
affine_mean_path = os.path.join(muscle_dir, "affine_mean.nrrd")
bspline_mean_path = os.path.join(muscle_dir, "bspline_mean.nrrd")

In [17]:
# ---- STEP 2: Rigid Registration ----
ref = distance_maps[ref_index]
print("Starting rigid registration...")
rigid_transforms = [rigid_register(ref, img) for img in tqdm(distance_maps, desc="Rigid Registration")]
rigid_aligned = [apply_transform(img, ref, tfm) for img, tfm in zip(distance_maps, rigid_transforms)]
rigid_mean = masked_mean(rigid_aligned, clip_val=clip_val)
sitk.WriteImage(rigid_mean, rigid_mean_path)

Starting rigid registration...


Rigid Registration: 100%|██████████| 101/101 [06:03<00:00,  3.60s/it]


In [18]:
# ---- STEP 3: Affine Registration ----
affine_transforms = [affine_register(rigid_mean, img) for img in tqdm(rigid_aligned, desc="Affine Registration")]
affine_aligned = [apply_transform(img, rigid_mean, tfm) for img, tfm in zip(rigid_aligned, affine_transforms)]
affine_mean = masked_mean(affine_aligned, clip_val=clip_val)
sitk.WriteImage(affine_mean, affine_mean_path)

Affine Registration: 100%|██████████| 101/101 [11:14<00:00,  6.68s/it]


In [ ]:
# ---- STEP 4: BSpline Registration ----
bspline_transforms = []

for i, img in enumerate(tqdm(affine_aligned, desc="BSpline Registration")):
    
    # 1) Overlap check (your existing filter)
    if not check_overlap(affine_mean, img):
        print(f"⚠️ Image {i}: Skipped (poor overlap). Using identity.")
        bspline_transforms.append(sitk.Transform(3, sitk.sitkIdentity))
        continue
    
    # 2) Try BSpline registration
    try:
        tfm = bspline_register(affine_mean, img)
        bspline_transforms.append(tfm)

    except Exception as e:
        # Catch all BSpline failures (spacing=0, degenerate axis, grid collapse, etc.)
        print(f"❌ Image {i}: BSpline failed. Using identity.")
        print(f"   Reason: {e}")
        bspline_transforms.append(sitk.Transform(3, sitk.sitkIdentity))

# ---- Apply transforms ----
bspline_aligned = [
    apply_transform(img, affine_mean, tfm)
    for img, tfm in zip(affine_aligned, bspline_transforms)
]

# ---- Compute template ----
template = masked_mean(bspline_aligned, clip_val=clip_val)
sitk.WriteImage(template, bspline_mean_path)

print("\nBSpline stage complete.")
print(f"Skipped or identity transforms: {sum(1 for t in bspline_transforms if t.GetName() == 'IdentityTransform')}")


BSpline Registration:   0%|          | 0/101 [00:00<?, ?it/s]WARNING: In /tmp/SimpleITK-build/ITK/Modules/Numerics/Optimizersv4/src/itkLBFGSBOptimizerv4.cxx, line 99
LBFGSBOptimizerv4 (0x1318f2810): LBFGSB optimizer does not support scaling. All scales are set to one.

BSpline Registration:   1%|          | 1/101 [01:15<2:05:48, 75.48s/it]WARNING: In /tmp/SimpleITK-build/ITK/Modules/Numerics/Optimizersv4/src/itkLBFGSBOptimizerv4.cxx, line 99
LBFGSBOptimizerv4 (0x131de8b10): LBFGSB optimizer does not support scaling. All scales are set to one.

BSpline Registration:   2%|▏         | 2/101 [02:40<2:14:15, 81.37s/it]WARNING: In /tmp/SimpleITK-build/ITK/Modules/Numerics/Optimizersv4/src/itkLBFGSBOptimizerv4.cxx, line 99
LBFGSBOptimizerv4 (0x3474f53e0): LBFGSB optimizer does not support scaling. All scales are set to one.

BSpline Registration:   3%|▎         | 3/101 [03:54<2:06:54, 77.69s/it]WARNING: In /tmp/SimpleITK-build/ITK/Modules/Numerics/Optimizersv4/src/itkLBFGSBOptimizerv4.cxx, li

### Saving template to binary and distance transform

In [ ]:
bspline_mean_path = os.path.join(muscle_dir, "bspline_mean.nrrd")

template = sw.Image(bspline_mean_path)
template_binary = template.binarize(0, 500)

# Template distance transform
iso_value = 0.0
sigma     = 1.3
antialias_iterations = 30

template_dt = template_binary.copy().antialias(antialias_iterations).resample([1.5, 1.5, 1.5], sw.InterpolationType.NearestNeighbor)
template_dt = template_dt.computeDT(iso_value)

# Template mesh
highres_points = 2000
template_mesh = template_binary.copy().antialias(antialias_iterations).toMesh(1e-6).smooth(iterations=30, relaxation=0.1).remesh(highres_points, adaptivity=0.0)

# Save the groomed template
template_binary.write(os.path.join(muscle_dir, f"{muscle_name}_template_bin.nrrd"))
template_dt.write(os.path.join(muscle_dir, f"{muscle_name}_template_dt.nrrd"))
template_mesh.write(os.path.join(muscle_dir, f"{muscle_name}_template_mesh.ply"))

{
number of points: 2068,
number of faces: 4132,
center: [50.0994, 54.858, 178.151],
center or mass: [50.0284, 56.8735, 170.642],
bounding box: {
	min: [22.6925, 20.4551, 38.204],
	max: [77.5062, 89.261, 318.098]
},
field names: 
	Normals

# Grooming

In [ ]:
# Load and initial grooming
shape_extention = '.nrrd'
random.seed(41)  # For reproducibility

# Files to process
all = True
num_samples = 20
left = True  # Process left segmentations
right = True # Process right segmentations
randomize = True  # Randomize the order of files

# File names for the next step
bin_extension = '.nrrd'
all_files = sorted(glob.glob(os.path.join(bin_dir, '*' + bin_extension)))
print(f"Total number of binary files found: {len(all_files)}")

if all:
    binary_filenames = all_files
else:
    # Filter by left/right based on filename
    left_files = [f for f in all_files if "_left" in os.path.basename(f)] if left else []
    right_files = [f for f in all_files if "_right" in os.path.basename(f)] if right else []

    # Calculate how many to take from each
    if left and right:
        each_side = num_samples // 2
        selected_left = random.sample(left_files, min(each_side, len(left_files)))
        selected_right = random.sample(right_files, min(each_side, len(right_files)))
        binary_filenames = selected_left + selected_right

    elif left:
        binary_filenames = random.sample(left_files, min(num_samples, len(left_files)))

    elif right:
        binary_filenames = random.sample(right_files, min(num_samples, len(right_files)))

    else:
        raise ValueError("At least one of 'left' or 'right' must be True.")


# Initialize lists for storing segments and names
shape_seg_list = []
shape_names = []

for filename in binary_filenames:
    seg_filename = filename.split('/')[-1]
    shape_name = os.path.splitext(seg_filename)[0].replace('_bin', '')
    
    print(f"Processing: {shape_name}")
    shape_seg = sw.Image(filename)
    shape_seg_list.append(shape_seg)
    shape_names.append(shape_name)
    
print(f"Total segments processed: {len(shape_seg_list)}")

Total number of binary files found: 202
Processing: MOTION001_left
Processing: MOTION001_right
Processing: MOTION002_left
Processing: MOTION002_right
Processing: MOTION003_left
Processing: MOTION003_right
Processing: MOTION004_left
Processing: MOTION004_right
Processing: MOTION005_left
Processing: MOTION005_right
Processing: MOTION006_left
Processing: MOTION006_right
Processing: MOTION007_left
Processing: MOTION007_right
Processing: MOTION008_left
Processing: MOTION008_right
Processing: MOTION009_left
Processing: MOTION009_right
Processing: MOTION010_left
Processing: MOTION010_right
Processing: MOTION011_left
Processing: MOTION011_right
Processing: MOTION012_left
Processing: MOTION012_right
Processing: MOTION013_left
Processing: MOTION013_right
Processing: MOTION014_left
Processing: MOTION014_right
Processing: MOTION015_left
Processing: MOTION015_right
Processing: MOTION016_left
Processing: MOTION016_right
Processing: MOTION017_left
Processing: MOTION017_right
Processing: MOTION018_lef

## Rigid Alignment

In [ ]:
reg_iterations = 50
highres_points = 2000
iso_value = 0.0
antialias_iterations = 20

shape_mesh_list = []
aligned_seg_list = []
aligned_mesh_list = []
transforms = []

# Directories
mesh_left_dir = os.path.join(mesh_dir, "left/")
mesh_right_dir = os.path.join(mesh_dir, "right/")
rig_left_dir = os.path.join(mesh_rig_dir, "left/")
rig_right_dir = os.path.join(mesh_rig_dir, "right/")

os.makedirs(rig_left_dir, exist_ok=True)
os.makedirs(rig_right_dir, exist_ok=True)

# Load and prepare the reference
# bspline_mean_path = os.path.join(muscle_dir, f"{muscle_name}_template.nrrd")
bspline_mean_path = os.path.join(muscle_dir, "affine_mean.nrrd")
reference = sw.Image(bspline_mean_path).binarize(0, 500).antialias(antialias_iterations)
ref_mesh = reference.toMesh(0.0).smooth(iterations=30, relaxation=0.1).remesh(highres_points, adaptivity=0.0)

for i, seg in tqdm(enumerate(shape_seg_list), total=len(shape_seg_list), desc="Processing Shapes"):
    seg = seg.copy().antialias(antialias_iterations)

    # 🧊 Pad the segmentation to prevent post-transform cropping
    seg = seg.pad(100, 0.0)

    # 🧱 Mesh the (padded) segment
    mesh = seg.toMesh(1e-6).smooth(iterations=30, relaxation=0.1).remesh(highres_points, adaptivity=0.0)
    if "left" in shape_names[i]:
        output_filename = os.path.join(mesh_left_dir, f"{shape_names[i]}.ply")
    else:
        output_filename = os.path.join(mesh_right_dir, f"{shape_names[i]}.ply")

    shape_mesh_list.append(mesh.copy())
    mesh.write(output_filename)

    # 🧭 Compute similarity transform
    transform_matrix = mesh.createTransform(ref_mesh, sw.Mesh.AlignmentType.Rigid, reg_iterations)

    # 🧱 Apply transform to the mesh
    mesh.applyTransform(transform_matrix)

    # 💾 Save outputs
    aligned_mesh_list.append(mesh)
    transforms.append(transform_matrix)

    if "left" in shape_names[i]:
        out_mesh_filename = os.path.join(rig_left_dir, f"{shape_names[i]}.ply")
    else:
        out_mesh_filename = os.path.join(rig_right_dir, f"{shape_names[i]}.ply")

    mesh.write(out_mesh_filename)

print(f"Processed {len(aligned_mesh_list)} meshes")


Processing Shapes: 100%|██████████| 202/202 [03:54<00:00,  1.16s/it]

Processed 202 meshes


In [ ]:
# Save to JSON
with open(os.path.join(mesh_rig_dir + 'transforms.json'), 'w') as f:
    json.dump([t.tolist() for t in transforms], f)

## Volume - Normalized

In [ ]:
# --- Compute all volumes first ---
volumes = []
original_volume_records = []

for seg, name in zip(shape_seg_list, shape_names):
    # ORIGINAL MESH (before smoothing / remeshing / scaling / rigid)
    mesh_orig = seg.copy().antialias(antialias_iterations).toMesh(iso_value)
    vol_orig = compute_mesh_volume(mesh_orig)

    volumes.append(vol_orig)

    # Save name + volume
    original_volume_records.append({
        "shape_name": name,
        "original_volume_mm3": vol_orig
    })

# Save original volumes to CSV
volume_df = pd.DataFrame(original_volume_records)
volume_csv_path = os.path.join(muscle_dir, f"{muscle_name}_original_volumes.csv")
volume_df.to_csv(volume_csv_path, index=False)

print(f"💾 Saved original volumes to: {volume_csv_path}")

💾 Saved original volumes to: data/semitendinosus/semitendinosus_original_volumes.csv


In [ ]:
reg_iterations = 50
highres_points = 2000
iso_value = 0.0
antialias_iterations = 20

aligned_mesh_list = []
transforms = []

# Directories
mesh_left_dir = os.path.join(mesh_dir, "left/")
mesh_right_dir = os.path.join(mesh_dir, "right/")
novol_left_dir = os.path.join(mesh_novol_dir, "left/")
novol_right_dir = os.path.join(mesh_novol_dir, "right/")

os.makedirs(mesh_left_dir, exist_ok=True)
os.makedirs(mesh_right_dir, exist_ok=True)
os.makedirs(novol_left_dir, exist_ok=True)
os.makedirs(novol_right_dir, exist_ok=True)

# Load and prepare the reference
# bspline_mean_path = os.path.join(muscle_dir, f"{muscle_name}_template.nrrd")
bspline_mean_path = os.path.join(muscle_dir, "affine_mean.nrrd")
reference = sw.Image(bspline_mean_path).binarize(0, 500).antialias(antialias_iterations)
ref_mesh = reference.toMesh(0.0).smooth(iterations=30, relaxation=0.1).remesh(highres_points, adaptivity=0.0)

def compute_mesh_volume(mesh):
    """
    Compute signed volume of a closed triangular mesh in mm^3.
    """
    pts = mesh.points()
    faces = mesh.faces()
    v1 = pts[faces[:, 0]]
    v2 = pts[faces[:, 1]]
    v3 = pts[faces[:, 2]]
    vol = np.sum(np.einsum('ij,ij->i', v1, np.cross(v2, v3))) / 6.0
    return abs(vol)

# --- Compute all volumes first ---
volumes = []
original_volume_records = []

for seg, name in zip(shape_seg_list, shape_names):
    # ORIGINAL MESH (before smoothing / remeshing / scaling / rigid)
    mesh_orig = seg.copy().antialias(antialias_iterations).toMesh(iso_value)
    vol_orig = compute_mesh_volume(mesh_orig)

    volumes.append(vol_orig)

    # Save name + volume
    original_volume_records.append({
        "shape_name": name,
        "original_volume_mm3": vol_orig
    })

# Save original volumes to CSV
volume_df = pd.DataFrame(original_volume_records)
volume_csv_path = os.path.join(muscle_dir, f"{muscle_name}_original_volumes.csv")
volume_df.to_csv(volume_csv_path, index=False)

print(f"💾 Saved original volumes to: {volume_csv_path}")

target_volume = np.exp(np.mean(np.log(volumes)))  # geometric mean
print(f"📏 Target (geometric mean) volume: {target_volume:.2f} mm³")

# --- Determine target volume ---
target_volume = np.exp(np.mean(np.log(volumes)))  # geometric mean
ref_volume = compute_mesh_volume(ref_mesh)
scale_factor = (target_volume / ref_volume)**(1/3)
centeroid = ref_mesh.center()
ref_mesh.translate(-centeroid)
ref_mesh.scale([scale_factor]*3)
ref_mesh.translate(centeroid)
ref_mesh.write(os.path.join(muscle_dir, f"{muscle_name}_template_novol.ply"))

for i, seg in tqdm(enumerate(shape_seg_list), total=len(shape_seg_list), desc="Processing Shapes"):
    seg = seg.copy().pad(100, 0.0).antialias(antialias_iterations)
    mesh = seg.toMesh(iso_value).smooth(iterations=30, relaxation=0.1).remesh(highres_points, adaptivity=0.0)

    mesh_volume = compute_mesh_volume(mesh)
    scale_factor = (target_volume / mesh_volume)**(1/3)

    # Scale uniformly around centroid
    centroid = mesh.center()
    mesh.translate(-centroid)
    mesh.scale([scale_factor]*3)
    mesh.translate(centroid)

    # Continue with rigid registration...
    transform_matrix = mesh.createTransform(ref_mesh, sw.Mesh.AlignmentType.Rigid, reg_iterations)
    mesh.applyTransform(transform_matrix)

    # 💾 Save outputs
    aligned_mesh_list.append(mesh)
    transforms.append(transform_matrix)

    if "left" in shape_names[i]:
        out_mesh_filename = os.path.join(novol_left_dir, f"{shape_names[i]}.ply")
    else:
        out_mesh_filename = os.path.join(novol_right_dir, f"{shape_names[i]}.ply")

    mesh.write(out_mesh_filename)

print(f"Processed {len(aligned_mesh_list)} meshes")


💾 Saved original volumes to: data/vastus_lateralis/vastus_lateralis_original_volumes.csv
📏 Target (geometric mean) volume: 641489.47 mm³


Processing Shapes: 100%|██████████| 202/202 [06:00<00:00,  1.79s/it]

Processed 202 meshes


In [ ]:
# Save to JSON
with open(os.path.join(mesh_novol_dir + 'transforms.json'), 'w') as f:
    json.dump([t.tolist() for t in transforms], f)

## Check mesh quality

In [45]:
# Load and initial grooming
random.seed(41)  # For reproducibility

# File names for the next step
extension = '.ply'
left_dir = os.path.join(mesh_novol_dir, "left/")
right_dir = os.path.join(mesh_novol_dir, "right/")
all_files = sorted(glob.glob(os.path.join(left_dir, '*' + extension)))
print(f"Total number of binary files found: {len(all_files)}")

# Initialize lists for storing segments and names
mesh_list = []

for filename in all_files:
    seg_filename = filename.split('/')[-1]
    shape_seg = sw.Mesh(filename)
    mesh_list.append(shape_seg)
    
print(f"Total segments processed: {len(mesh_list)}")
# sw.plot_meshes(mesh_list, use_same_window=True, notebook=False, show_borders=True)

Total number of binary files found: 101
Total segments processed: 101


In [ ]:
sw.plot_meshes([mesh_list[0], mesh_list[3], mesh_list[4]],
                use_same_window=False,
                notebook=False,
                show_borders=False,
                meshes_color='lightpink',
                show_bounds = False,
                show_all_edges=False,
                show_axes=False
                )

In [ ]:
import trimesh

def check_mesh_quality(mesh_dir):
    mesh_paths = sorted([os.path.join(mesh_dir, f) for f in os.listdir(mesh_dir) if f.endswith('.ply')])

    issues = []
    print(f"\nChecking {len(mesh_paths)} meshes in: {mesh_dir}\n")

    for path in mesh_paths:
        name = os.path.basename(path)
        try:
            mesh = trimesh.load(path, force='mesh')
            watertight = mesh.is_watertight
            manifold = mesh.is_watertight and mesh.is_winding_consistent
            components = len(mesh.split(only_watertight=False))

            print(f"✔️ {name} | Watertight: {watertight} | Manifold: {manifold} | Components: {components}")

            if not watertight or not manifold or components > 1:
                issues.append((name, watertight, manifold, components))

        except Exception as e:
            print(f"❌ Failed to load {name}: {e}")
            issues.append((name, "error", "error", "error"))

    if issues:
        print("\n⚠️ Issues found in the following meshes:")
        for name, watertight, manifold, components in issues:
            print(f"  - {name}: watertight={watertight}, manifold={manifold}, components={components}")
    else:
        print("\n✅ All meshes passed the quality checks!")

# Example usage
check_mesh_quality(rig_right_dir)
check_mesh_quality(rig_left_dir)
check_mesh_quality(novol_right_dir)
check_mesh_quality(novol_left_dir)


Checking 101 meshes in: data/semimembranosus/mesh_novolume/right/

✔️ MOTION001_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION002_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION003_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION004_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION005_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION006_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION007_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION008_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION009_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION010_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION011_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION012_right.ply | Watertight: True | Manifold: True | Components: 1
✔️ MOTION013_right.ply | Waterti

In [36]:
def check_mesh_volume(mesh_dir, reference=None, summary=True):
    """
    Check and summarize the volume (mm³) of all meshes in a directory.

    Parameters
    ----------
    mesh_dir : str
        Path to folder containing .ply meshes.
    reference : float or None
        If provided, compute volume ratio to this reference (e.g., template volume).
    summary : bool
        If True, print mean, std, min, max, etc.
    """
    mesh_paths = sorted([os.path.join(mesh_dir, f) for f in os.listdir(mesh_dir) if f.endswith('.ply')])

    volumes = []
    print(f"\n📦 Checking volume of {len(mesh_paths)} meshes in: {mesh_dir}\n")

    for path in mesh_paths:
        name = os.path.basename(path)
        try:
            mesh = trimesh.load_mesh(path, process=False)
            vol = abs(mesh.volume)
            volumes.append(vol)

            if reference:
                ratio = vol / reference
                print(f"✔️ {name:<30} | Volume: {vol:10.2f} mm³ | Ratio: {ratio:6.3f}")
            else:
                print(f"✔️ {name:<30} | Volume: {vol:10.2f} mm³")

        except Exception as e:
            print(f"❌ Failed to load {name}: {e}")

    if not volumes:
        print("\n⚠️ No valid meshes found or failed to load volumes.")
        return None

    vols = np.array(volumes)

    if summary:
        print("\n📊 Summary Statistics")
        print(f"  Mean Volume   : {vols.mean():.2f} mm³")
        print(f"  Std Dev       : {vols.std():.2f} mm³")
        print(f"  Median Volume : {np.median(vols):.2f} mm³")
        print(f"  Min Volume    : {vols.min():.2f} mm³")
        print(f"  Max Volume    : {vols.max():.2f} mm³")

        if reference:
            ratios = vols / reference
            print(f"  Mean Ratio    : {ratios.mean():.3f}")
            print(f"  Std Ratio     : {ratios.std():.3f}")

    return vols

# 1️⃣ Check volumes in a directory
volumes = check_mesh_volume(novol_left_dir)
volumes = check_mesh_volume(novol_right_dir)

# 3️⃣ Get geometric mean for normalization later
geom_mean_vol = np.exp(np.mean(np.log(volumes)))
print(f"\n📏 Geometric mean volume: {geom_mean_vol:.2f} mm³")


📦 Checking volume of 101 meshes in: data/semimembranosus/mesh_novolume/left/

✔️ MOTION001_left.ply             | Volume:  221494.40 mm³
✔️ MOTION002_left.ply             | Volume:  221494.40 mm³
✔️ MOTION003_left.ply             | Volume:  221494.42 mm³
✔️ MOTION004_left.ply             | Volume:  221494.42 mm³
✔️ MOTION005_left.ply             | Volume:  221494.41 mm³
✔️ MOTION006_left.ply             | Volume:  221494.41 mm³
✔️ MOTION007_left.ply             | Volume:  221494.41 mm³
✔️ MOTION008_left.ply             | Volume:  221494.41 mm³
✔️ MOTION009_left.ply             | Volume:  221494.41 mm³
✔️ MOTION010_left.ply             | Volume:  221494.41 mm³
✔️ MOTION011_left.ply             | Volume:  221494.41 mm³
✔️ MOTION012_left.ply             | Volume:  221494.42 mm³
✔️ MOTION013_left.ply             | Volume:  221494.41 mm³
✔️ MOTION014_left.ply             | Volume:  221494.41 mm³
✔️ MOTION015_left.ply             | Volume:  221494.42 mm³
✔️ MOTION016_left.ply             | 